# Generate high resolution tidal forcing file by resampling

In [38]:
# get environment variables
import os
import sys
from dotenv import load_dotenv, find_dotenv

# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()

# load up the entries as environment variables
load_dotenv(dotenv_path)

sys.path.append(os.environ.get('srcdir'))

# always reload modules marked with "%aimport"
%load_ext autoreload
%autoreload 1

from features.resample import low_to_high_frc
%aimport features.resample

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load grids, low res tide file and set up output path

In [14]:
import xarray as xr
import matplotlib.pyplot as plt

hr_grd_path = os.path.join(os.environ.get("prodir"),'waom5_grd.nc')
hr_grd = xr.open_dataset(hr_grd_path)

lr_grd_path = os.path.join(os.environ.get("prodir"),'waom10_grd.nc')
lr_grd = xr.open_dataset(lr_grd_path)

lr_frc_path =  os.path.join(os.environ.get("prodir"),'waom10_tds.nc')
lr_frc = xr.open_dataset(lr_frc_path)

hr_frc_path = os.path.join(os.environ.get("prodir"),'waom5_tds.nc')

In [31]:
lr_frc

<xarray.Dataset>
Dimensions:             (eta_rho: 730, string: 4, tide_period: 13, xi_rho: 860)
Coordinates:
  * tide_period         (tide_period) timedelta64[ns] 0 days 12:25:14.163600 ...
Dimensions without coordinates: eta_rho, string, xi_rho
Data variables:
    lat_rho             (eta_rho, xi_rho) float64 ...
    lon_rho             (eta_rho, xi_rho) float64 ...
    mask_rho            (eta_rho, xi_rho) float64 ...
    tide_Cphase         (tide_period, eta_rho, xi_rho) float64 ...
    tide_Cangle         (tide_period, eta_rho, xi_rho) float64 ...
    tide_Cmin           (tide_period, eta_rho, xi_rho) float64 ...
    tide_Cmax           (tide_period, eta_rho, xi_rho) float64 ...
    tidal_constituents  (string, tide_period) |S1 ...
    tide_Pphase         (tide_period, eta_rho, xi_rho) float64 ...
    tide_Pamp           (tide_period, eta_rho, xi_rho) float64 ...
Attributes:
    Type:       ROMS Tidal Forcing File
    Title:      Forcing for WAOM10  domain
    grid_file:  /home/ub

## Drop all varaibles, but include tide_period variable

In [39]:
hr_frc = lr_frc.drop(lr_frc.variables)
hr_frc.coords['tide_period'] = lr_frc.tide_period
hr_frc

<xarray.Dataset>
Dimensions:      (tide_period: 13)
Coordinates:
  * tide_period  (tide_period) timedelta64[ns] 0 days 12:25:14.163600 ...
Data variables:
    *empty*
Attributes:
    Type:       ROMS Tidal Forcing File
    Title:      Forcing for WAOM10  domain
    grid_file:  /home/ubuntu/bigStick/tidal_melting/data/preprocessing/proces...
    Source:     OTPS-http://volkov.oce.orst.edu/tides/global.html
    history:    Wed Feb 21 11:49:17 2018: ncrename -O -v tide_Eamp,tide_Pamp ...

## Resample phase and amplitude fields and add to high res tide file 

In [40]:
# interpolate 2D variables
hr_frc['tide_Pamp'] = low_to_high_frc(lr_frc.tide_Pamp,lr_grd,hr_grd,'rho',lr_frc.tide_period);
hr_frc['tide_Pphase'] = low_to_high_frc(lr_frc.tide_Pphase,lr_grd,hr_grd,'rho',lr_frc.tide_period);

set up empty hr data array
Fill in the mask of lr data, resample to high resolution grid and fill hr mask with fill value:  0.0
processing time:  0
processing time:  1
processing time:  2
processing time:  3
processing time:  4
processing time:  5
processing time:  6
processing time:  7
processing time:  8
processing time:  9
processing time:  10
processing time:  11
processing time:  12
set up empty hr data array
Fill in the mask of lr data, resample to high resolution grid and fill hr mask with fill value:  0.0
processing time:  0
processing time:  1
processing time:  2
processing time:  3
processing time:  4
processing time:  5
processing time:  6
processing time:  7
processing time:  8
processing time:  9
processing time:  10
processing time:  11
processing time:  12


In [43]:
hr_frc

<xarray.Dataset>
Dimensions:      (eta_rho: 1460, tide_period: 13, xi_rho: 1720)
Coordinates:
  * tide_period  (tide_period) timedelta64[ns] 0 days 12:25:14.163600 ...
  * eta_rho      (eta_rho) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 ...
  * xi_rho       (xi_rho) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 ...
Data variables:
    tide_Pamp    (tide_period, eta_rho, xi_rho) float64 0.1337 0.1337 0.1337 ...
    tide_Pphase  (tide_period, eta_rho, xi_rho) float64 190.2 190.2 190.2 ...
Attributes:
    Type:       ROMS Tidal Forcing File
    Title:      Forcing for WAOM10  domain
    grid_file:  /home/ubuntu/bigStick/tidal_melting/data/preprocessing/proces...
    Source:     OTPS-http://volkov.oce.orst.edu/tides/global.html
    history:    Wed Feb 21 11:49:17 2018: ncrename -O -v tide_Eamp,tide_Pamp ...

## Write out high res tidal forcing file

In [ ]:
hr_frc.to_netcdf(hr_frc_path,unlimited_dims='tide_period')